# ML for climate and Energy (MEC51057)
# Group project: Weather station

**Mentor**:
- Julio 

**Students**:
- Dimitri Iratchet
- Fabien Lagnieu
- Tristan Waddington


# "clean notebook"

In [ ]:
# imports
import os
from pathlib import Path


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

--- 
## 1. Project presentation

---
## 2. Data Loading / preparation / study

---
## 3. Prediction of Paris Weather
Supervised regression

### 3.0 Baseline

### 3.1 Model 1
linear

### 3.2 Model 2


### 3.3 Model 3
Dimension reduction

### 3.4 Model 4 
Deep learning

---
## 4. Detection of anomalies
unsupervised classification

## 5. Conclusion

About the dataset
About the project and objectives
About the team organisation

If we had to do it again:
- keep:
- change: